Don't forget to login to Ollama, download the CLI if you haven't already, and add your public key!

In [7]:
import json

def process_merges(obj):
    """
    Recursively searches for 'merges' lists in a JSON-like structure and flattens them.

    :param obj: The JSON object (dict, list, or other types).
    :return: The modified JSON object with 'merges' lists processed.
    """
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == "merges" and isinstance(value, list):
                obj[key] = [" ".join(pair) for pair in value if isinstance(pair, list)]
            else:
                process_merges(value)
    elif isinstance(obj, list):
        for item in obj:
            process_merges(item)

def flatten_merges_in_json(json_file, output_file=None):
    """
    Reads a JSON file, processes all 'merges' lists to flatten them,
    and writes the updated JSON to a new file or returns it.

    :param json_file: Path to the input JSON file.
    :param output_file: Optional path for the output JSON file.
    :return: Processed JSON as a dictionary if output_file is not provided.
    """
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        process_merges(data)
        if output_file:
            with open(output_file, 'w', encoding='utf-8') as file:
                json.dump(data, file, indent=4, ensure_ascii=False)
            print(f"Processed JSON has been saved to {output_file}")
        else:
            return data
    except Exception as e:
        print(f"An error occurred: {e}")


Processed JSON has been saved to ../models/SFW_merged/tokenizer.json


In [ ]:
modelfile_content = """
FROM ../SFW_merged

ADAPTER checkpoint-150

PARAMETER temperature 0.7
PARAMETER top_p 0.7
PARAMETER stop "<|im_end|>"
PARAMETER stop "<eos>"
PARAMETER stop "<end_of_turn>"
"""

with open("../models/SFW/Modelfile", "w") as file:
    file.write(modelfile_content)

In [ ]:
modelfile_content = """
FROM ../NSFW_merged

ADAPTER checkpoint-150

PARAMETER temperature 0.7
PARAMETER top_p 0.7
PARAMETER stop "<|im_end|>"
PARAMETER stop "<eos>"
PARAMETER stop "<end_of_turn>"
"""

with open("../models/NSFW/Modelfile", "w") as file:
    file.write(modelfile_content)

In [ ]:
flatten_merges_in_json("../models/SFW_merged/tokenizer.json", "../models/SFW_merged/tokenizer.json")

In [ ]:
flatten_merges_in_json("../models/NSFW_merged/tokenizer.json", "../models/NSFW_merged/tokenizer.json")

In [ ]:
import os

os.system("cd ../models/SFW && ollama create your-username/cookiebaker-sfw -f Modelfile")
os.system("ollama push your-username/cookiebaker-sfw")

In [ ]:
os.system("cd ../models/NSFW && ollama create your-username/cookiebaker-nsfw -f Modelfile")
os.system("ollama push your-username/cookiebaker-nsfw")